In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import random
import string
from collections import defaultdict
from itertools import product, chain
import numpy as np
from pattern.en import comparative

In [146]:
import torch
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from enum import Enum
from typing import List, Optional, Union

from child_frames import frames
from utils import *

import sys
sys.path.insert(0, '/nas/xd/projects/transformers/src/transformers')

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, GlueDataset
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import HfArgumentParser, Trainer, TrainingArguments, set_seed
from transformers import RobertaForMaskedLM, RobertaTokenizer

logging.basicConfig(level=logging.ERROR)

In [158]:
tokenizer.tokenize(' unrelated')

['Ġunrelated']

In [167]:
A_template = "{rel_prefix} {dt} {ent0} {rel} {dt} {ent1} {rel_suffix}"
B_templates = ["{pred_prefix} {dt} {ent} {pred}", "{pred_prefix} {pred} {dt} {ent}"]
B_template = B_templates[0]
entailment_templates = [
    "{A} ? {conj} , {B} .",  # yes/no/maybe
    "{A} , so {B} ? {conj} .",
]
marker = '*'
def get_comparative(word, add_marker=True):
    compar = comparative(word)
    if add_marker:
        compar = compar.replace('more ', 'more %s ' % marker) if compar.startswith('more ') else marker + ' ' + compar
    return compar
    
def negate_sent(sent):
    assert ' is ' in sent
    neg_sents = []
    neg_sents.append(sent.replace(' is ', ' is not '))
    neg_sents.append('it is unlikely that ' + sent)
    return neg_sents

def make_sentences(index=-1, entities=["X", "Z"], entity_set=string.ascii_uppercase, determiner="",
                   relation_group=[["big",], ["small"]], rand_relation_group=[["short"], ["tall", "high"]],
                   relation_prefix="", relation_suffix="", predicate_prefix="",
                   n_entity_trials=3, has_negA=True, has_negB=True, has_neutral=False, mask_types=['sent_rel'], 
                   lexical_relations=['same', 'opposite', 'unrelated'], tag_lexical_rel=False, tag_entity_rel=False):
    def form_As(relations):
        return [A_template.format(dt=determiner, ent0=ent0, ent1=ent1, rel=rel, rel_prefix=relation_prefix, rel_suffix=relation_suffix)
              for ent0, ent1, rel in [entities + relations[:1], reverse(entities) + reverse(relations)[:1]]]

    As = []
    for rel0 in relation_group[0]:
        for rel1 in relation_group[1]:
            relations = ["is %s:%d than" % (get_comparative(rel), i) for i, rel in enumerate([rel0, rel1])]
            As += form_As(relations)
    As = list(set(As))
    negAs = join_lists([negate_sent(A)[:1] for A in As]) if has_negA else []

    def form_Bs(predicates): 
        f = mask if 'entity' in mask_types else (lambda x: x)
        return [B_template.format(dt=determiner, ent=f(ent), pred=pred, pred_prefix=predicate_prefix)
              for ent, pred in zip(entities, predicates)]

    Bs, negBs = {'orig': [], 'rand': []}, {}
    for k, group in zip(['orig', 'rand'], [relation_group, rand_relation_group]):
        for rel0 in group[0]:
            for rel1 in group[1]:
                predicates = ["is %s:%d" % (get_comparative(rel), i) for i, rel in enumerate([rel0, rel1])]
                Bs[k] += form_Bs(predicates)
    for k in Bs:
        Bs[k] = list(set(Bs[k]))
        if has_negB:
            negBs[k] = join_lists([negate_sent(B)[:1] for B in Bs[k]])
            Bs[k], negBs[k] = Bs[k] + [swap_entities(negB) for negB in negBs[k]], negBs[k] + [swap_entities(B) for B in Bs[k]]
        else:
            negBs[k] = [swap_entities(B) for B in Bs[k]]

    def form_sentences(sentence_template, As, Bs, conj):
        def extract_rel_id(s): return int(s[s.index(':') + 1])
        def get_lexical_rel(rel_id_A, rel_id_B): return 'same' if rel_id_A == rel_id_B else 'opposite'
        def strip_rel_id(s, lexical_rel=''):
            rel_id_span = s[s.index(':'): s.index(':') + 2]
            if lexical_rel != '':
                if 'lexical_rel' in mask_types: lexical_rel = mask(lexical_rel)
                lexical_rel = ' ( ' + lexical_rel + ' )'
            return s.replace(rel_id_span, lexical_rel)
        def compare_and_tag_entity(B, A):
            entity = [e for e in entities if e in B][0]
            entity_rel = 'former' if A.strip().startswith(entity) else 'latter'
            if 'entity_rel' in mask_types: entity_rel = mask(entity_rel)
            return B.replace(entity, entity + ' ( ' + entity_rel + ' )')                    
        
        if 'sent_rel' in mask_types: conj = mask(conj)
        As_with_rel_ids = [(A, extract_rel_id(A)) for A in As]
        Bs_with_rel_ids = [(B, extract_rel_id(B)) for B in Bs]
            
        sentences = []
        for (A, rel_id_A), (B, rel_id_B) in product(As_with_rel_ids, Bs_with_rel_ids):
            lexical_rel = 'unrelated' if 'Maybe' in conj else get_lexical_rel(rel_id_A, rel_id_B)
            if lexical_rel in lexical_relations:
                if tag_entity_rel: B = compare_and_tag_entity(B, A)
                if not tag_lexical_rel: lexical_rel = ''
                sent = sentence_template.format(A=strip_rel_id(A), B=strip_rel_id(B, lexical_rel), conj=conj)
                sent = " " + " ".join(sent.split())
                sentences.append(sent)
        return sentences

    sentences = defaultdict(list)
    for entailment_template in entailment_templates[-1:]:
        for A, B, conj in [(As, Bs['orig'], 'Right'), 
                           (negAs, negBs['orig'], 'Right'), 
                           (As, negBs['orig'], 'Wrong'), 
                           (negAs, Bs['orig'], 'Wrong'),
                           (As, Bs['rand'], 'Maybe'), 
                           (negAs, negBs['rand'], 'Maybe'), 
                           (As, negBs['rand'], 'Maybe'), 
                           (negAs, Bs['rand'], 'Maybe'),
                          ]:
            sentences[conj] += form_sentences(entailment_template, A, B, conj)
    assert len(sentences['Right']) == len(sentences['Wrong']), '%d %d' % (len(sentences['Right']), len(sentences['Wrong']))
    if has_neutral: sentences['Maybe'] = random.sample(sentences['Maybe'], len(sentences['Right']))
    sentences = join_lists(sentences[k] for k in (sentences.keys() if has_neutral else ['Right', 'Wrong']))
    
    substituted_sent_groups = []
    for sent in sentences:
        sent_group = []
        for _ in range(n_entity_trials):
            e0, e1 = random.sample(entity_set, 2)
            sent_group.append(sent.replace(entities[0], e0).replace(entities[1], e1))
        substituted_sent_groups.append(sent_group)
    return sentences, substituted_sent_groups

make_sentences(has_negA=True, has_negB=True, has_neutral=True, tag_lexical_rel=True, tag_entity_rel=True, 
               mask_types=['sent_rel', 'lexical_rel', 'entity_rel'])[0]

[' Z is * smaller than X , so X ( [ latter ] ) is * bigger ( [ opposite ] ) ? [ Right ] .',
 ' Z is * smaller than X , so Z ( [ former ] ) is * smaller ( [ same ] ) ? [ Right ] .',
 ' Z is * smaller than X , so Z ( [ former ] ) is not * bigger ( [ opposite ] ) ? [ Right ] .',
 ' Z is * smaller than X , so X ( [ latter ] ) is not * smaller ( [ same ] ) ? [ Right ] .',
 ' X is * bigger than Z , so X ( [ former ] ) is * bigger ( [ same ] ) ? [ Right ] .',
 ' X is * bigger than Z , so Z ( [ latter ] ) is * smaller ( [ opposite ] ) ? [ Right ] .',
 ' X is * bigger than Z , so Z ( [ latter ] ) is not * bigger ( [ same ] ) ? [ Right ] .',
 ' X is * bigger than Z , so X ( [ former ] ) is not * smaller ( [ opposite ] ) ? [ Right ] .',
 ' Z is not * smaller than X , so X ( [ latter ] ) is not * bigger ( [ opposite ] ) ? [ Right ] .',
 ' Z is not * smaller than X , so Z ( [ former ] ) is not * smaller ( [ same ] ) ? [ Right ] .',
 ' Z is not * smaller than X , so Z ( [ former ] ) is * bigger ( [ 

In [63]:
model_class, tokenizer_class, shortcut = RobertaForMaskedLM, RobertaTokenizer, 'roberta-large'
model, tokenizer = None, tokenizer_class.from_pretrained(shortcut)

In [165]:
random.shuffle(frames)
all_lines = [make_sentences(relation_group=rg, rand_relation_group=frames[(i + 1) % len(frames)], n_entity_trials=3, 
                            has_negA=True, has_negB=True, tag_lexical_rel=True, tag_entity_rel=False,
                            has_neutral=True, mask_types=['lexical_rel'])[1] 
             for i, rg in enumerate(frames)]
# all_lines = join_lists(all_lines)
# all_lines = join_lists(all_lines)
for k in CHILDDataset.all_lines: CHILDDataset.all_lines[k] = None
train_dataset = CHILDDataset(all_lines, tokenizer, max_noise_len=0, split_pct=[0.7, 0.3, 0.0], mode='train')
eval_dataset = CHILDDataset(all_lines, tokenizer, max_noise_len=0, split_pct=[0.7, 0.3, 0.0], mode='dev')
print('nTrain = %d, nValid = %d' % (len(train_dataset), len(eval_dataset)))

in convert_example_to_features: features.labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 5483, -100, -100, -100, -100, -100, -100, -100, -100, -100]
in convert_example_to_features: features.labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 276, -100, -100, -100, -100, -100, -100, -100, -100, -100]
nTrain = 16524, nValid = 6156


In [166]:
model = model_class.from_pretrained('roberta-base', model=model)

training_args = TrainingArguments(output_dir="./models/model_name", 
    overwrite_output_dir=True, do_train=True, do_eval=True,
    per_device_train_batch_size=32, per_device_eval_batch_size=64,
    learning_rate=2e-5, num_train_epochs=3,
    logging_steps=100, eval_steps=100, save_steps=0,
    evaluate_during_training=True,
)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset)
trainer.tokenizer = tokenizer
trainer.train()

{'loss': 0.531, 'learning_rate': 1.8710509348807224e-05, 'epoch': 0.193, 'step': 100}



{'eval_loss': 0.236, 'eval_acc0': 0.951, 'eval_stat0': 'Ġsame 0.38 0.99, Ġopposite 0.29 0.99, Ġunrelated 0.33 1.00, ', 'epoch': 0.193, 'step': 100}
{'loss': 0.045, 'learning_rate': 1.7421018697614445e-05, 'epoch': 0.387, 'step': 200}



{'eval_loss': 0.434, 'eval_acc0': 0.967, 'eval_stat0': 'Ġsame 0.34 1.00, Ġopposite 0.33 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 0.387, 'step': 200}
{'loss': 0.012, 'learning_rate': 1.6131528046421664e-05, 'epoch': 0.58, 'step': 300}



{'eval_loss': 0.283, 'eval_acc0': 0.97, 'eval_stat0': 'Ġsame 0.34 1.00, Ġopposite 0.33 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 0.58, 'step': 300}
{'loss': 0.01, 'learning_rate': 1.4842037395228886e-05, 'epoch': 0.774, 'step': 400}



{'eval_loss': 0.549, 'eval_acc0': 0.961, 'eval_stat0': 'Ġsame 0.35 1.00, Ġopposite 0.32 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 0.774, 'step': 400}
{'loss': 0.01, 'learning_rate': 1.3552546744036106e-05, 'epoch': 0.967, 'step': 500}


...


{'eval_loss': 0.337, 'eval_acc0': 0.975, 'eval_stat0': 'Ġsame 0.33 1.00, Ġopposite 0.33 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 2.515, 'step': 1300}
{'loss': 0.0, 'learning_rate': 1.9471308833010964e-06, 'epoch': 2.708, 'step': 1400}



{'eval_loss': 0.326, 'eval_acc0': 0.977, 'eval_stat0': 'Ġsame 0.33 1.00, Ġopposite 0.34 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 2.708, 'step': 1400}
{'loss': 0.0, 'learning_rate': 6.576402321083172e-07, 'epoch': 2.901, 'step': 1500}



{'eval_loss': 0.328, 'eval_acc0': 0.976, 'eval_stat0': 'Ġsame 0.33 1.00, Ġopposite 0.33 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 2.901, 'step': 1500}





{'eval_loss': 0.328, 'eval_acc0': 0.976, 'eval_stat0': 'Ġsame 0.33 1.00, Ġopposite 0.33 1.00, Ġunrelated 0.33 1.00, ', 'epoch': 3.0, 'step': 1551}


TrainOutput(global_step=1551, training_loss=0.040174619352948214)

In [128]:
dataloader = trainer.get_eval_dataloader()

In [129]:
for inputs in dataloader: break

In [130]:
inputs

{'input_ids': tensor([[  0, 387,  16,  ...,   2,   0,   0],
         [  0, 534,  16,  ...,   2,   0,   0],
         [  0, 975,  16,  ...,   2,   0,   0],
         ...,
         [  0, 487,  16,  ..., 479,   2,   0],
         [  0, 673,  16,  ..., 479,   2,   0],
         [  0, 725,  16,  ..., 479,   2,   0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         ...,
         [-100, -1

In [133]:
inputs = trainer._prepare_inputs(inputs, model)
loss, logits = model(**inputs)

In [134]:
probs = F.softmax((logits * (inputs['labels'] != -100).unsqueeze(-1)).sum(dim=1), dim=-1)

In [144]:
bsz, seq_len, vocab_size = (logits).size()
masks = (inputs['labels'] != -100)
n_mask = masks.sum(dim=-1)[0].item()
pred_labels = logits.masked_select(mask.unsqueeze(-1)).view(bsz, n_mask, vocab_size).argmax(dim=-1)
labels = inputs['labels'].masked_select(masks).view(bsz, n_mask)

torch.Size([64, 3])

In [132]:
inputs['labels'][0]

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 5442,
        -100, -100, -100, -100, -100, 5483, -100, -100, 5143, -100, -100, -100,
        -100])

In [108]:
pred_labels = (logits * (inputs['labels'] != -100).unsqueeze(-1)).sum(dim=1).argmax(dim=-1)

labels = (inputs['labels'] * (inputs['labels'] != -100)).sum(dim=-1)

(pred_labels == labels).float().mean().item()

0.90625

In [35]:
print('%.1f' % 94.433)

94.4


In [33]:
for i, label_id in enumerate(pred_labels):
    print(tokenizer._convert_id_to_token(label_id.item()), probs[i, label_id].item())

ĠRight 0.6260414719581604
ĠRight 0.6174842119216919
ĠRight 0.627226710319519
ĠRight 0.6074517965316772
ĠRight 0.6034893989562988
ĠRight 0.5928120613098145
ĠMaybe 0.9999963045120239
ĠMaybe 0.9999955892562866
ĠMaybe 0.9999963045120239
ĠRight 0.6209385395050049
ĠRight 0.6153814196586609
ĠRight 0.6189284324645996
ĠRight 0.6173365712165833
ĠRight 0.6170286536216736
ĠRight 0.6148619651794434
ĠMaybe 0.9999971389770508
ĠMaybe 0.9999972581863403
ĠMaybe 0.9999969005584717
ĠRight 0.6081119179725647
ĠRight 0.6188942790031433
ĠRight 0.6106655597686768
ĠRight 0.6149485111236572
ĠRight 0.6151918172836304
ĠRight 0.6127419471740723
ĠRight 0.6444377303123474
ĠRight 0.6497629284858704
ĠRight 0.6418505311012268
ĠRight 0.6264133453369141
ĠRight 0.6273400783538818
ĠRight 0.6344654560089111
ĠRight 0.6158509254455566
ĠRight 0.6200970411300659
ĠRight 0.6208559274673462
ĠMaybe 0.9999988079071045
ĠMaybe 0.9999986886978149
ĠMaybe 0.9999988079071045
ĠMaybe 0.9999974966049194
ĠMaybe 0.9999971389770508
ĠMaybe 0.9999

In [21]:
pred_labels.bincount().topk(3)

torch.return_types.topk(
values=tensor([40, 24,  0], device='cuda:0'),
indices=tensor([5143, 5359,    0], device='cuda:0'))

In [22]:
labels.bincount().topk(3)

torch.return_types.topk(
values=tensor([24, 22, 18], device='cuda:0'),
indices=tensor([ 5359,  5143, 31273], device='cuda:0'))

In [26]:
tokenizer._convert_id_to_token(5359)

'ĠMaybe'

In [46]:
values, indices = probs.topk(5, dim=-1)

In [52]:
for top_idx in indices:
    print(top_idx.tolist())
    print(tokenizer.convert_ids_to_tokens(top_idx.tolist()))

[5143, 31273, 13984, 235, 37234]
['ĠRight', 'ĠWrong', 'Right', 'Ġright', 'ĠCorrect']
[5143, 31273, 235, 13984, 10039]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'ĠLeft']
[5143, 31273, 235, 13984, 38103]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'ĠRIGHT']
[5143, 31273, 235, 13984, 38103]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'ĠRIGHT']
[5143, 31273, 235, 13984, 38103]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'ĠRIGHT']
[31273, 5143, 235, 13984, 38103]
['ĠWrong', 'ĠRight', 'Ġright', 'Right', 'ĠRIGHT']
[31273, 5143, 235, 13984, 38103]
['ĠWrong', 'ĠRight', 'Ġright', 'Right', 'ĠRIGHT']
[5143, 31273, 235, 13984, 1593]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'Ġwrong']
[5143, 31273, 13984, 235, 38103]
['ĠRight', 'ĠWrong', 'Right', 'Ġright', 'ĠRIGHT']
[5143, 31273, 13984, 235, 37234]
['ĠRight', 'ĠWrong', 'Right', 'Ġright', 'ĠCorrect']
[5143, 31273, 13984, 235, 37234]
['ĠRight', 'ĠWrong', 'Right', 'Ġright', 'ĠCorrect']
[5143, 31273, 235, 13984, 38103]
['ĠRight', 'ĠWrong', 'Ġright', 'Right', 'ĠRIGHT']
